### Make sure to install ChromeDriver here (May need to update which chromedriver is saved in current working python directory):
https://sites.google.com/a/chromium.org/chromedriver/downloads

https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab

In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import datetime
#from datetime import date
#import datetime

from requests import get
from requests.exceptions import RequestException
from contextlib import closing

In [3]:
time_of_search = str(datetime.now())

In [ ]:
import datetime

# Will need to change aug_days_num to 31 and sep_days_num to 30

aug_days_num = 1
sep_days_num = 0

date_list = []

for dayofmonth in range(1,aug_days_num+1):
    date = datetime.date(2019, 8, dayofmonth)
    date = str(date)
    date_list.append(date)
    
for dayofmonth in range(1,sep_days_num+1):
    date = datetime.date(2019, 9, dayofmonth)
    date = str(date)
    date_list.append(date)

In [ ]:
def get_from_divs(soup, driver, city, origin):
    """
    Accepts a soup object and returns the various features of flights as a dataframe.    
    """
    soup = BeautifulSoup(driver.page_source, 'lxml')
    """Extract flight times"""
    time_divs = soup.find_all('div', attrs={'class': 'gws-flights-results__times'})
    
    
    if time_divs is not None:
        deptimes = []
        for time_div in time_divs:
            #print(time_div)
            deptimes.append(time_div.getText().split('–')[0].strip())
        
        arrtimes = []
        for time_div in time_divs:
            arrtimes.append(time_div.getText().split('–')[1].strip())
    else:
        raise Exception('No flight times were found.')
    
    
    """Extract carrier"""
    carrier_divs = soup.find_all('div', attrs={'class': 'gws-flights-results__carriers'})
    if carrier_divs is not None:
        carriers = []
        for carrier_div in carrier_divs:
            carriers.append(carrier_div.getText().strip())
    else:
        raise Exception('No flight carriers were found.')
    
    
    """Extract duration of flights"""
    duration_divs = soup.find_all('div', attrs={'class': 'gws-flights-results__duration'})
    if duration_divs is not None:
        durations = []
        for duration_div in duration_divs:
            durations.append(duration_div.getText().strip())
    
    else:
        raise Exception('No flight durations were found.')
    
    """Extract price of flights"""
    price_divs = soup.find_all('div', attrs={'class': 'gws-flights-results__itinerary-price'})
    if price_divs is not None:
        prices = []
        for price_div in price_divs:
            prices.append(price_div.getText().strip())
            #prices = prices[1::2]
    else:
        raise Exception('No flight prices were found')  
    prices = prices[1::2]
    
    df = pd.DataFrame({
    "time_of_search": time_of_search,
    "origin" : origin,
    "destination" : city,
    "date" : date,
    "price": prices,
    "currency": "USD",
    "deptime": deptimes,
    "arrtime": arrtimes,
    "duration": durations,
    "carrier": carriers
    })
    
    return df

In [ ]:
'''Setting up parameters'''
origin = 'SFO'
destination_list = ['MIA', 'LAX'] 
airlines = 'AA,DL,UA'

In [ ]:
# First make empty dataframe objects // or else clear flight dataframes
for city in destination_list:
    df_city = 'df_' + city
    vars()[df_city] = pd.DataFrame()
    
    df_city_yest = 'df_'+city+'_yest'
    vars()[df_city_yest] = pd.DataFrame()


In [23]:
url = f'https://www.lme.com/en-GB/Metals/Non-ferrous/Nickel#tabIndex=0'
soup, driver = simple_get(url)

soup = BeautifulSoup(driver.page_source, 'lxml')
time_divs = soup.find_all('div', attrs={'class': 'gws-flights-results__times'})
print(time_divs)    


[]


WebDriverException: Message: chrome not reachable
  (Session info: chrome=78.0.3904.108)


In [17]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome("chromedriver")
driver.implicitly_wait(20)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
'''
driver.implicitly_wait(20)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')
if type(driver) == webdriver.chrome.webdriver.WebDriver:
    return soup, driver
else:
    return None

'''

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 79


In [14]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    beautifulsoup object.
    """
    try:
        #with closing(get(url, stream=True)) as resp:
        chrome_options = webdriver.ChromeOptions()
        driver = webdriver.Chrome("chromedriver")
        driver.implicitly_wait(20)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        if type(driver) == webdriver.chrome.webdriver.WebDriver:
            return soup, driver
        else:
            return None

    except Exception:
        # MAIN ERROR? 
        #log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        print('Error retrieving contents at {}'.format(url))
        return None

In [ ]:

            # Save yesterday's dataframe as a csv
            vars()[df_city_yest] = vars()[df_city]
            vars()[df_city_yest].to_csv(df_city_yest+'.csv', index=False)
            
            # Save today's updated dataframe as a csv / Replace yesterday's
            vars()[df_city] = vars()[df_city].append(df)
            vars()[df_city].to_csv(df_city+'.csv', index=False)
            


### Testing